In [109]:
from typing import Any, Literal
import random
import math

In [110]:
M = 7
R = 4

In [ ]:
class Hamming:
    @staticmethod
    def add_control_bits(record: list[Literal[0,1]]) -> None:
        new_record = []
        curr_record_i = 0
        curr_level = 0
        for i in range(2*len(record)):
            if curr_record_i == len(record):
                break
            if (2**curr_level-1) == i:
                new_record.append(0)
                curr_level += 1
            else:
                new_record.append(record[curr_record_i])
                curr_record_i += 1

        # print(new_record)

        curr_level = 0
        end_curr_level = math.ceil(math.log2(len(new_record)))
        for curr_level in range(end_curr_level):
            new_record[2**curr_level-1] = sum([sum(new_record[i:i+2**curr_level]) for i in range(2**curr_level-1, len(new_record), 2*(2**curr_level))]) % 2
            # print(curr_level, [[i, i+2**curr_level] for i in range(2**curr_level-1, len(new_record), 2*(2**curr_level))])

        return new_record
        
    
    @staticmethod
    def brake_random_bit(record: list[Literal[0,1]], break_place: int | None = None) -> None:
        new_record = record.copy()
        if break_place is None:
            break_place = random.randint(0, len(new_record)-1)
        new_record[break_place] = 1 - new_record[break_place]
        return new_record


    @staticmethod
    def fix_bit(record: list[Literal[0,1]]) -> None:
        new_record = record.copy()
        variant = 0

        curr_level = 0
        end_curr_level = math.ceil(math.log2(len(new_record)))
        for curr_level in range(end_curr_level):
            val = sum([sum(new_record[i:i+2**curr_level]) for i in range(2**curr_level-1, len(new_record), 2*(2**curr_level))]) % 2
            # print(curr_level, [[i, i+2**curr_level] for i in range(2**curr_level-1, len(new_record), 2*(2**curr_level))])
            if val:
                variant += 2**curr_level

        if variant:
            new_record[variant-1] = 1 - new_record[variant-1]
        return new_record
    

    @staticmethod
    def remove_control_bits(record: list[Literal[0,1]]) -> None:
        new_record = []
        for i in range(len(record)):
            if 2**(int(math.log2(i+1))) == i+1:
                continue
            new_record.append(record[i])
        return new_record




    @staticmethod
    def split(record: list[Literal[0,1]]) -> list[list[Literal[0,1]]]:
        records = []
        for i in range(math.ceil(len(record)/M)):
            cur = record[M*i:M*i+M]
            if len(cur) != M:
                cur += [0 for _ in range(M-len(cur))]
            records.append(cur)
        return records

    @staticmethod
    def encode(records: list[list[Literal[0,1]]]) -> None:
        return [Hamming.add_control_bits(record) for record in records]
    
    @staticmethod
    def breaking(records: list[list[Literal[0,1]]]) -> None:
        return [Hamming.brake_random_bit(record) for record in records]
    
    @staticmethod
    def fix(records: list[list[Literal[0,1]]]) -> None:
        return [Hamming.fix_bit(record) for record in records]
    
    @staticmethod
    def decode(records: list[list[Literal[0,1]]]) -> None:
        return [Hamming.remove_control_bits(record) for record in records]
    
    @staticmethod
    def join(records: list[list[Literal[0,1]]]) -> list[list[Literal[0,1]]]:
        result = []
        for record in records:
            result += record
        return result
        


In [112]:
M = 7
R = 7

In [113]:
if 2**R < M + R + 1:
    print("нельзя использовать такую комбинацию")
else:
    print(f"делаем ({M+R},{M})")

делаем (14,7)


In [114]:
def get_str_message(message: list[Literal[0,1]], is_list: bool = False) -> str:
    if is_list:
        return [" ".join([str(i) for i in msg]) for msg in message]
    return " ".join([str(i) for i in message])

def demonstrate(message: str) -> None:
    message = list(map(int, message))
    splitted_message = Hamming.split(message)
    encoded_message = Hamming.encode(splitted_message)
    encoded_message_broken = Hamming.breaking(encoded_message)
    encoded_message_fixed = Hamming.fix(encoded_message_broken)
    decoded_message = Hamming.fix(encoded_message_fixed)
    message_from_start = Hamming.join(decoded_message)

    print(f"{get_str_message(message)} - исходная последовательность")
    print(f"{get_str_message(splitted_message, True)} - делёная последовательность")
    print(f"{get_str_message(encoded_message, True)} - добавили контрольные биты")
    print(f"{get_str_message(encoded_message_broken, True)} - поломали последовательность")
    print(f"{get_str_message(encoded_message_fixed, True)} - починили последовательность")
    print(f"{get_str_message(decoded_message, True)} - удалили контрольные биты и получили исходную последовательность")
    print(f"{get_str_message(message_from_start)} - вернулись к изначальному сообщению")

In [115]:
demonstrate("10011010")


2
8
1
8
1 0 0 1 1 0 1 0 - исходная последовательность
['1 0 0 1 1 0 1', '0 0 0 0 0 0 0'] - делёная последовательность
['0 1 1 1 0 0 1 0 1 0 1', '0 0 0 0 0 0 0 0 0 0 0'] - добавили контрольные биты
['0 1 1 1 0 0 1 0 1 1 1', '0 0 0 0 0 0 0 0 1 0 0'] - поломали последовательность
['0 1 1 1 0 0 1 0 1 0 1', '0 0 0 0 0 0 0 0 0 0 0'] - починили последовательность
['0 1 1 1 0 0 1 0 1 0 1', '0 0 0 0 0 0 0 0 0 0 0'] - удалили контрольные биты и получили исходную последовательность
0 1 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 - вернулись к изначальному сообщению


In [116]:
demonstrate("0100010000111101")

1
8
1
2
2
0 1 0 0 0 1 0 0 0 0 1 1 1 1 0 1 - исходная последовательность
['0 1 0 0 0 1 0', '0 0 0 1 1 1 1', '0 1 0 0 0 0 0'] - делёная последовательность
['1 1 0 1 1 0 0 1 0 1 0', '1 1 0 1 0 0 1 1 1 1 1', '1 0 0 1 1 0 0 0 0 0 0'] - добавили контрольные биты
['1 1 0 1 1 0 0 1 1 1 0', '1 1 1 1 0 0 1 1 1 1 1', '1 1 0 1 1 0 0 0 0 0 0'] - поломали последовательность
['1 1 0 1 1 0 0 1 0 1 0', '1 1 0 1 0 0 1 1 1 1 1', '1 0 0 1 1 0 0 0 0 0 0'] - починили последовательность
['1 1 0 1 1 0 0 1 0 1 0', '1 1 0 1 0 0 1 1 1 1 1', '1 0 0 1 1 0 0 0 0 0 0'] - удалили контрольные биты и получили исходную последовательность
1 1 0 1 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 - вернулись к изначальному сообщению
